In [1]:
from dataloader import dataset_pipeline
from torch.utils.data import Dataset, DataLoader

/Users/kieraguan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import argparse
import os

# make a name for this experiment
exp_name = 'project'  # change it as you like!

# make a directory for the experiment
if not os.path.exists(exp_name):
    os.makedirs(exp_name)
parser = argparse.ArgumentParser(description=exp_name)

# add hyperparameters to the parser
parser.add_argument('--batch-size', type=int, default=2000,
                    help='input batch size for training (default: 2000)')
parser.add_argument('--epochs', type=int, default=100,
                    help='number of epochs to train (default: 100)')
parser.add_argument('--freq-dim', type=int, default=24,
                    help='the frequency dimension (default:24)')
parser.add_argument('--cuda', action='store_true', default=False,
                    help='enables CUDA training (default: False)')  # when you have a GPU
parser.add_argument('--lr', type=float, default=4e-4,
                    help='learning rate (default: 4e-4)')
parser.add_argument('--model-save', type=str,  default='best_model.pt',
                    help='path to save the best model')
parser.add_argument('--tr-data', type=str,  default='tr.hdf5',
                    help='path to training dataset')
parser.add_argument('--val-data', type=str,  default='val.hdf5',
                    help='path to validation dataset')
parser.add_argument('--test-data', type=str,  default='test.hdf5',
                    help='path to testing dataset')

args, _ = parser.parse_known_args()
args.cuda = args.cuda and torch.cuda.is_available()

if args.cuda:
    kwargs = {'num_workers': 1, 'pin_memory': True} 
else:
    kwargs = {}



In [8]:
train_loader = DataLoader(dataset_pipeline('Data/tr_data.hdf5'), 
                          batch_size=args.batch_size, 
                          shuffle=True, 
                          **kwargs)
validation_loader = DataLoader(dataset_pipeline('Data/val_data.hdf5'),
                               batch_size=args.batch_size, 
                               shuffle=False, 
                               **kwargs)
test_loader = DataLoader(dataset_pipeline('Data/test_data.hdf5'), 
                         batch_size=args.batch_size, 
                         shuffle=False, 
                         **kwargs)
args.dataset_len = len(train_loader)
args.log_step = args.dataset_len // 4

In [9]:
#model
from model import MLP
import torch.optim as optim
model_MLP = MLP()
print(model_MLP)

# activate GPU if you have it
if args.cuda:
    model_MLP = model_MLP.cuda()
# define the optimizer
optimizer = optim.Adam(model_MLP.parameters(), lr=args.lr)
# the scheduler is to define a exponential decay on the learning rate
# every time the scheduler is activated, the learning rate is decayed by the given ratio
# this is used to adjust the learning rate when the training plateaus 
scheduler  =optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
scheduler.step()

MLP(
  (layer1): Sequential(
    (0): Linear(in_features=24, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (activation1): ReLU()
  (norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer2): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Linear(in_features=128, out_features=5, bias=True)
    (1): Softmax()
  )
)


In [11]:
from model import CNN_rnn
import torch.optim as optim
model_crnn =CNN_rnn()
print(model_crnn)

# activate GPU if you have it
if args.cuda:
    model_crnn = model_crnn.cuda()
# define the optimizer
optimizer = optim.Adam(model_crnn.parameters(), lr=args.lr)
# the scheduler is to define a exponential decay on the learning rate
# every time the scheduler is activated, the learning rate is decayed by the given ratio
# this is used to adjust the learning rate when the training plateaus 
#scheduler  =optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
scheduler  =optim.lr_scheduler.StepLR(optimizer,step_size=11, gamma=0.5)
scheduler.step()

CNN_rnn(
  (cov1): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1))
  (cov2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (cov3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (cov4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (cov5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, strid

In [12]:
from loss_func import loss_func
from classcify import class_accuracy
import time
from train import train,validate

In [ ]:
training_loss = []
validation_loss = []

decay_cnt = 0
for epoch in range(1, args.epochs + 1):
    if args.cuda:
        model_crnn.cuda()
    training_loss.append(train(model_crnn, epoch,train_loader,args.cuda,optimizer,args.log_step))
    validation_loss.append(validate(model_crnn, epoch,validation_loader,args.cuda))
    
    if training_loss[-1] == np.min(training_loss):
        print('      Best training model found.')
    if validation_loss[-1] == np.min(validation_loss):
        # save current best model
        with open(args.model_save, 'wb') as f:
            torch.save(model_crnn.cpu().state_dict(), f)
            print('      Best validation model found and saved.')
    
    print('-' * 99)
    decay_cnt += 1
    # lr decay
    # decay when no best training model is found for 3 consecutive epochs
    if np.min(training_loss) not in training_loss[-3:] and decay_cnt >= 3:
        scheduler.step()
        decay_cnt = 0
        print('      Learning rate decreased.')
        print('-' * 99)